# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [70]:
import json

In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [28]:
# Import data from .csv file
csvFile = os.path.join("/Users/juanvalderrama/Desktop/usc-virt-data-pt-03-2021-u-c/06-Python-APIs/Homework/output_data/cities.csv")
df = pd.read_csv(csvFile)

# for i, row in df.iterrows():
#     print(row[2])
df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,wukari,7.8500,9.7833,72.79,86,74,3.53,NG,1624484177
1,1,ushuaia,-54.8000,-68.3000,37.02,80,20,3.44,AR,1624483891
2,2,dikson,73.5069,80.5464,35.04,92,100,12.88,RU,1624484177
3,3,port alfred,-33.5906,26.8910,59.50,81,98,11.83,ZA,1624484178
4,4,hobart,-42.8794,147.3294,51.22,88,0,10.36,AU,1624484178
...,...,...,...,...,...,...,...,...,...,...
590,590,kruisfontein,-34.0033,24.7314,56.88,91,100,6.04,ZA,1624484423
591,591,uthal,25.8072,66.6219,82.92,79,58,7.81,PK,1624484424
592,592,rize,40.9044,40.8949,52.07,83,57,0.92,TR,1624484425
593,593,ugoofaaru,5.6667,73.0000,82.15,77,100,13.56,MV,1624484425


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
# Store latitude and longitude in locations
locations = df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = df["Humidity"]

In [12]:
# Configure gmaps
gmaps.configure(api_key=g_key)
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.
* A max temperature lower than 80 degrees but higher than 70.
* Wind speed less than 10 mph.
* Zero cloudiness.
* Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.



In [40]:
# Narrow down cities that fit criteria and drop any results with null values
new_df = df.loc[(df["Max Temp"] < 80) & (df["Max Temp"] > 70) 
                                    & (df["Wind Speed"] < 10) 
                                    & (df["Cloudiness"] == 0)].dropna()
new_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
26,26,qunduz,36.7290,68.8570,78.48,14,0,4.05,AF,1624484187
27,27,port said,31.2565,32.2841,77.36,71,0,5.53,EG,1624484188
78,78,baherden,38.4362,57.4316,77.88,17,0,4.68,TM,1624484208
81,81,gat,31.6100,34.7642,76.82,79,0,3.09,IL,1624484210
98,98,can,40.0333,27.0524,74.08,86,0,1.05,TR,1624484216
238,238,turkan,40.3639,50.2150,77.18,47,0,2.30,AZ,1624484274
303,303,orahovac,42.3994,20.6547,75.06,54,0,2.04,XK,1624484306
319,319,pervomayskoye,43.9190,46.7102,70.70,67,0,9.95,RU,1624484313
323,323,korla,41.7597,86.1469,71.49,21,0,8.55,CN,1624484315
356,356,nurota,40.5614,65.6886,73.49,24,0,8.70,UZ,1624484327


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [117]:
hotel_df = new_df
hotel_df['Hotel Name'] = ""
hotel_df.drop('Hotel', inplace=True, axis=1)
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
26,26,qunduz,36.7290,68.8570,78.48,14,0,4.05,AF,1624484187,
27,27,port said,31.2565,32.2841,77.36,71,0,5.53,EG,1624484188,
78,78,baherden,38.4362,57.4316,77.88,17,0,4.68,TM,1624484208,
81,81,gat,31.6100,34.7642,76.82,79,0,3.09,IL,1624484210,
98,98,can,40.0333,27.0524,74.08,86,0,1.05,TR,1624484216,
238,238,turkan,40.3639,50.2150,77.18,47,0,2.30,AZ,1624484274,
303,303,orahovac,42.3994,20.6547,75.06,54,0,2.04,XK,1624484306,
319,319,pervomayskoye,43.9190,46.7102,70.70,67,0,9.95,RU,1624484313,
323,323,korla,41.7597,86.1469,71.49,21,0,8.55,CN,1624484315,
356,356,nurota,40.5614,65.6886,73.49,24,0,8.70,UZ,1624484327,


In [126]:
params = {
    "radius": 5000,
    #"types": "lodging",
    #"rankby": "distance",
    "types": "hotel",
    "key": g_key
}
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    try:
        #hotel_df[row]['Hotel Name'] = results[1]['name']
        hotel_df.loc[index, 'Hotel Name'] = results[1]['name']
    except:
        hotel_df[row]['Hotel Name'] = None
    
  
   
    #print(row['Hotel Name'])
#     from pprint import pprint
#     #print(results)
#     print(json.dumps(results, indent=4, sort_keys=True))


In [127]:
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
26,26,qunduz,36.7290,68.8570,78.48,14,0,4.05,AF,1624484187,Jubaili Bros
27,27,port said,31.2565,32.2841,77.36,71,0,5.53,EG,1624484188,Aracan Hotel
78,78,baherden,38.4362,57.4316,77.88,17,0,4.68,TM,1624484208,TM CELL
81,81,gat,31.6100,34.7642,76.82,79,0,3.09,IL,1624484210,Tel Aerni Tel Jet
98,98,can,40.0333,27.0524,74.08,86,0,1.05,TR,1624484216,Kale - Guven Tesisat
238,238,turkan,40.3639,50.2150,77.18,47,0,2.30,AZ,1624484274,Software development
303,303,orahovac,42.3994,20.6547,75.06,54,0,2.04,XK,1624484306,Te Alushi
319,319,pervomayskoye,43.9190,46.7102,70.70,67,0,9.95,RU,1624484313,Kosyakinskaya Srednyaya Obsheobrazavatel'naya ...
323,323,korla,41.7597,86.1469,71.49,21,0,8.55,CN,1624484315,Korla No.2 Elementary School
356,356,nurota,40.5614,65.6886,73.49,24,0,8.70,UZ,1624484327,Guest House Ruslan Nurata & Tours


In [129]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [131]:
# Add marker layer ontop of heat map

gmaps.marker_layer(locations, info_box_content=hotel_info)
# fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))